In [6]:
import pandas  as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from datetime import timedelta
import tensorflow as tf

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.12 from "e:\Projects\Usama Ikram\LSTM based classification\env\Scripts\python.exe"
  * The NumPy version is: "2.1.1"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed while importing _multiarray_umath: The specified module could not be found.


In [21]:
df = pd.read_csv('Imputed_m96_Progressor.csv')

In [23]:
def viscode_to_months(viscode):
    if viscode == 'bl':
        return 0
    elif viscode.startswith('m'):
        return int(viscode[1:])  # Convert 'mXX' to number of months
    
df['Months'] = df['VISCODE'].apply(viscode_to_months)

# Step 2: Sort the data by Patient ID and Months
df = df.sort_values(by=['PTID', 'Months'])
viscode_index = df.columns.get_loc('VISCODE')

# Insert 'Months' column at the position right after 'VISCODE'
df.insert(viscode_index + 1, 'Months', df.pop('Months'))
# Step 3: Select features and target variable
# Remove non-feature columns (PTID, VISCODE, etc.)
features = df.drop(columns=['PTID', 'VISCODE', 'Calculated Diagnosis'])

# Target is 'Calculated Diagnosis'
target = df['Calculated Diagnosis']
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

In [11]:
df

,PTID,VISCODE,Months,CDRSB,ADAS11,ADAS13,ADASQ4,MMSE,RAVLT_immediate,RAVLT_learning,...,EcogPtDivatt,EcogPtTotal,EcogSPMem,EcogSPLang,EcogSPVisspat,EcogSPPlan,EcogSPOrgan,EcogSPDivatt,EcogSPTotal,Calculated Diagnosis
0,002_S_0295,bl,0,0.000000,3.000000,4.000000,1.000000,28.000000,56.000000,5.000000,...,0.946487,1.023623,1.135182,1.083713,1.131010,1.016576,0.976817,0.977503,1.054367,CNs
1,002_S_0295,m06,6,0.000000,5.330000,6.330000,1.000000,28.000000,50.000000,7.000000,...,1.011747,1.026511,1.185862,1.006525,1.044206,1.025742,1.021633,1.037495,1.054806,CNs
2,002_S_0295,m12,12,0.000000,4.670000,5.670000,1.000000,30.000000,53.000000,10.000000,...,0.985921,1.025781,1.163892,1.036070,1.064586,1.042524,0.997455,1.026956,1.055454,CNs
3,002_S_0295,m18,18,0.000000,4.170000,5.670000,1.500000,29.500000,49.000000,8.500000,...,0.975817,1.025275,1.163750,1.042146,1.069793,1.051914,0.991613,1.008476,1.055178,CNs
4,002_S_0295,m24,24,0.000000,3.670000,5.670000,2.000000,29.000000,45.000000,7.000000,...,0.963675,1.024709,1.174956,1.048991,1.066163,1.056240,0.980130,0.997897,1.054919,CNs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30578,941_S_6803,m72,72,0.997617,10.509164,16.352609,5.761848,26.198423,34.653974,4.779806,...,3.183554,2.390091,2.795805,1.808741,1.777533,1.518968,1.237558,1.535085,1.915996,MCIs
30579,941_S_6803,m78,78,0.867200,6.716921,9.177599,2.947717,29.620267,37.521772,3.109639,...,3.189672,2.571133,2.377254,1.529986,1.710471,1.620265,2.119877,2.228112,2.119254,MCIs
30580,941_S_6803,m84,84,2.760001,10.249716,16.054889,7.850159,29.350465,33.865883,0.538609,...,1.849112,1.946852,2.254674,1.849332,1.852305,1.636661,1.427320,2.170922,1.882390,MCIs
30581,941_S_6803,m90,90,0.754309,10.511876,9.475303,3.253254,29.461123,41.040897,2.411145,...,2.088639,2.868091,2.643641,1.868005,-0.011413,0.909256,1.439720,2.999306,1.529986,MCIs


In [12]:
def create_sequences(features, target):
    X, y = [], []
    for patient_id in df['PTID'].unique():
        patient_data = features[df['PTID'] == patient_id]  # Features for the patient
        patient_target = target[df['PTID'] == patient_id].values[-1]  # Final target value for the patient
        X.append(patient_data)  # Append all 17 records as one sequence
        y.append(patient_target)  # Target is the last diagnosis for the patient
    return np.array(X), np.array(y)

sequence_length = 17  # Since each patient has 17 records
X, y = create_sequences(features, target)

In [13]:
print(X.shape)
print(y.shape)

(1799, 17, 31)
(1799,)


In [19]:
features['']

,Months,CDRSB,ADAS11,ADAS13,ADASQ4,MMSE,RAVLT_immediate,RAVLT_learning,RAVLT_forgetting,RAVLT_perc_forgetting,...,EcogPtOrgan,EcogPtDivatt,EcogPtTotal,EcogSPMem,EcogSPLang,EcogSPVisspat,EcogSPPlan,EcogSPOrgan,EcogSPDivatt,EcogSPTotal
0,0,0.000000,3.000000,4.000000,1.000000,28.000000,56.000000,5.000000,3.000000,23.076900,...,1.089821,0.946487,1.023623,1.135182,1.083713,1.131010,1.016576,0.976817,0.977503,1.054367
1,6,0.000000,5.330000,6.330000,1.000000,28.000000,50.000000,7.000000,2.000000,16.666700,...,1.014480,1.011747,1.026511,1.185862,1.006525,1.044206,1.025742,1.021633,1.037495,1.054806
2,12,0.000000,4.670000,5.670000,1.000000,30.000000,53.000000,10.000000,2.000000,14.285700,...,1.040376,0.985921,1.025781,1.163892,1.036070,1.064586,1.042524,0.997455,1.026956,1.055454
3,18,0.000000,4.170000,5.670000,1.500000,29.500000,49.000000,8.500000,2.000000,15.476200,...,1.045690,0.975817,1.025275,1.163750,1.042146,1.069793,1.051914,0.991613,1.008476,1.055178
4,24,0.000000,3.670000,5.670000,2.000000,29.000000,45.000000,7.000000,2.000000,16.666700,...,1.055672,0.963675,1.024709,1.174956,1.048991,1.066163,1.056240,0.980130,0.997897,1.054919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30578,72,0.997617,10.509164,16.352609,5.761848,26.198423,34.653974,4.779806,6.344202,67.050994,...,2.444912,3.183554,2.390091,2.795805,1.808741,1.777533,1.518968,1.237558,1.535085,1.915996
30579,78,0.867200,6.716921,9.177599,2.947717,29.620267,37.521772,3.109639,2.706481,28.476376,...,2.583748,3.189672,2.571133,2.377254,1.529986,1.710471,1.620265,2.119877,2.228112,2.119254
30580,84,2.760001,10.249716,16.054889,7.850159,29.350465,33.865883,0.538609,3.660119,56.386614,...,1.837829,1.849112,1.946852,2.254674,1.849332,1.852305,1.636661,1.427320,2.170922,1.882390
30581,90,0.754309,10.511876,9.475303,3.253254,29.461123,41.040897,2.411145,2.910998,56.771089,...,1.861491,2.088639,2.868091,2.643641,1.868005,-0.011413,0.909256,1.439720,2.999306,1.529986


In [18]:
feature = features[:-1]
feature

,Months,CDRSB,ADAS11,ADAS13,ADASQ4,MMSE,RAVLT_immediate,RAVLT_learning,RAVLT_forgetting,RAVLT_perc_forgetting,...,EcogPtOrgan,EcogPtDivatt,EcogPtTotal,EcogSPMem,EcogSPLang,EcogSPVisspat,EcogSPPlan,EcogSPOrgan,EcogSPDivatt,EcogSPTotal
0,0,0.000000,3.000000,4.000000,1.000000,28.000000,56.000000,5.000000,3.000000,23.076900,...,1.089821,0.946487,1.023623,1.135182,1.083713,1.131010,1.016576,0.976817,0.977503,1.054367
1,6,0.000000,5.330000,6.330000,1.000000,28.000000,50.000000,7.000000,2.000000,16.666700,...,1.014480,1.011747,1.026511,1.185862,1.006525,1.044206,1.025742,1.021633,1.037495,1.054806
2,12,0.000000,4.670000,5.670000,1.000000,30.000000,53.000000,10.000000,2.000000,14.285700,...,1.040376,0.985921,1.025781,1.163892,1.036070,1.064586,1.042524,0.997455,1.026956,1.055454
3,18,0.000000,4.170000,5.670000,1.500000,29.500000,49.000000,8.500000,2.000000,15.476200,...,1.045690,0.975817,1.025275,1.163750,1.042146,1.069793,1.051914,0.991613,1.008476,1.055178
4,24,0.000000,3.670000,5.670000,2.000000,29.000000,45.000000,7.000000,2.000000,16.666700,...,1.055672,0.963675,1.024709,1.174956,1.048991,1.066163,1.056240,0.980130,0.997897,1.054919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30577,66,0.290856,6.722127,10.288789,5.053795,28.234804,38.240977,5.231293,3.015937,39.830389,...,2.240115,2.423350,2.182211,2.931676,2.220693,2.084129,2.035756,2.408299,1.914647,2.456962
30578,72,0.997617,10.509164,16.352609,5.761848,26.198423,34.653974,4.779806,6.344202,67.050994,...,2.444912,3.183554,2.390091,2.795805,1.808741,1.777533,1.518968,1.237558,1.535085,1.915996
30579,78,0.867200,6.716921,9.177599,2.947717,29.620267,37.521772,3.109639,2.706481,28.476376,...,2.583748,3.189672,2.571133,2.377254,1.529986,1.710471,1.620265,2.119877,2.228112,2.119254
30580,84,2.760001,10.249716,16.054889,7.850159,29.350465,33.865883,0.538609,3.660119,56.386614,...,1.837829,1.849112,1.946852,2.254674,1.849332,1.852305,1.636661,1.427320,2.170922,1.882390


In [25]:
y = tf.keras.preprocessing.sequence.TimeseriesGenerator(
    features,
    target,
    length = 17,
    sampling_rate=1,s
    stride=1,
    start_index=0,
    end_index=None,
    shuffle=False,
    reverse=False,
    batch_size=128
)

In [31]:
y[0].shape

AttributeError: 'tuple' object has no attribute 'shape'